In [0]:
%tensorflow_version 1.x

import random
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf

In [2]:
!wget 'https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt'
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


--2019-12-04 07:31:09--  https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/laax0sxqzzlddtq/ferdosi.txt [following]
--2019-12-04 07:31:10--  https://www.dropbox.com/s/raw/laax0sxqzzlddtq/ferdosi.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7b8b52b331f88b32750f65d9a6.dl.dropboxusercontent.com/cd/0/inline/Atkqqbq0y_TC1nWdbVqrlB5ZAvlvJp7-85t0Yyx4HOCP1CnLPKxVaTYIXvmn1sx_Z-jfXgrYT5iXA0C7rHVvO2nyRqEwpGDFmF-NQl983iQzyQ/file# [following]
--2019-12-04 07:31:10--  https://uc7b8b52b331f88b32750f65d9a6.dl.dropboxusercontent.com/cd/0/inline/Atkqqbq0y_TC1nWdbVqrlB5ZAvlvJp7-85t0Yyx4HOCP1CnLPKxVaTYIXvmn1sx_Z-jfXgrYT5iXA0C7rHVvO2nyRqEwpGDFmF-NQl983iQzyQ/file
Resolving uc7b8b52b331

In [3]:
chars = sorted(list(set(text)))
len_of_char = len(chars)
print('total chars:', len(chars))
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

total chars: 37


In [4]:
max_length = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
print('nb sequences:', len(sentences))

nb sequences: 848363


In [5]:
print('Vectorization...')
x = np.zeros((len(sentences), max_length, len(chars)))
y = np.zeros((len(sentences)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char2idx[char]] = 1
    y[i] = char2idx[next_chars[i]]

    

Vectorization...


In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_length - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_length]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, max_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_char = idx2char[pred_id]

            generated += next_char
            sentence = sentence[1:] + next_char

        print(generated)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
model = keras.Sequential()
model.add(keras.layers.CuDNNLSTM(2048, input_shape=(max_length, len_of_char)))
# model.add(keras.layers.GRU(256, input_shape=(max_length, len_of_char)))
model.add(keras.layers.Dense(len_of_char, activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_2 (CuDNNLSTM)     (None, 2048)              17096704  
_________________________________________________________________
dense_3 (Dense)              (None, 37)                75813     
Total params: 17,172,517
Trainable params: 17,172,517
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
print_callback = keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
save_model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='ferdosi-char.h5', save_best_only='True', monitor='acc')
model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback, save_model_checkpoint])

Train on 848363 samples
Epoch 1/60
848256/848363 [============================>.] - ETA: 0s - loss: 1.7211 - acc: 0.5054
----- Generating text after Epoch: 0
----- diversity: 1.0
----- Generating with seed: "رسید
یکی بیکران ژرف "
رسید
یکی بیکران ژرف و کرد پپاسان جست
نبودش ببیآبش تنگاستان
گتنگای بودم ز لشکر ببخت
هم اندرد آتش به ژرف مغم
نگارش نسازی کنون رازین
فروبی و جان را ببین بلید
بخیره شد اسپ مخت پیچیر مید
وزین گونه را گفت تا پادرش
به تخت ششختا بترس و شرم
ز گرت خرد هسته یزدان بخاک
به بخشیدنی پوش منشند
بود گشت دورد پر آن برمسا
فزونی نمانم به موش پی
عگاهد به تو مرش تا بودگان
مگر مغزه منش سال تاو
ببیرم بود بود و داور تو آمد زیا
تنه کو لب خیر ن
848363/848363 [==============================] - 426s 503us/sample - loss: 1.7211 - acc: 0.5054
Epoch 2/60
173952/848363 [=====>........................] - ETA: 4:11 - loss: 1.4932 - acc: 0.5574